# AI Powered Recipe Recommendation System 

#### Introduction
In the world, where people are more health conscious and work driven, often face difficulty to prepare meals considering time, ingredients availability and dietary preferences. Although there are a millions of recipes available on internet, it requires efforts to find the one with fits with available resources. 

The objective of this project to utilize advanced technologies backed up with large dataset helps to generate customized meal suggestions based on user input.

#### Problem Statement:

The project's goal is to create a recipe recommendation system powered by AI that makes meal planning easier. A dataset containing 231638 recipes is filtered by ingredients, dietary restrictions, and time constraints to provide users with personalised food recommendations. Based on user choices, this system offers quick, customised meal selections.

### Rectangular dataset : Raw_Recipe.csv

In [ ]:
import pandas as pd 

df = pd.read_csv('./Dataset/RAW_recipes.csv')
df.head(3)

In [26]:
import pandas as pd

class RawRecipe:
    def __init__(self):
        self.file_path = './Dataset/RAW_recipes.csv'
        self.data = None
    
    # Loads the data from a CSV file.
    def load_data(self):
        self.data = pd.read_csv(self.file_path)
        print(f"---> STEP 1 : Loads the data from a CSV file. \r\n")
        print(f"RAW_recipes.csv : Data loaded successfully.")
        print(f"Total Records : {self.data.shape[0]} \r\n")
        return self.data
    
    # Data quality : Null Check
    def check_null_values(self):
        print(f"---> STEP 2 : Null Check for data \r\n")
        if self.data is not None:
            nulls = self.data.isnull().sum()
            print(nulls)
            return nulls
        else:
            print("Data not loaded.")

    # Summarize basic statistics
    def summarize_dataset(self):
        print(f"\r\n---> STEP 3 : Summarize basic statistics for columns \r\n")
        if self.data is not None:
            # Preparation time
            print("Data summary : Time to prepare the recepe in minutes \r\n")
            minutesSummary = self.data['minutes'].describe()
            print(f"Minimum \t:\t {minutesSummary['min']:.2f} minutes")
            print(f"Maximum \t:\t {minutesSummary['max']:.2f} minutes i.e {minutesSummary['max']/60:.2f} hours")
            print(f"Mean    \t:\t {minutesSummary['mean']:.2f} i.e {minutesSummary['mean']/60:.2f} hours")
            median_minute = self.data['minutes'].median()
            print(f"Median  \t:\t {median_minute:.2f} minutes i.e {median_minute/60:.2f} hours")
            mode_minute = self.data['minutes'].mode()
            print(f"Mode    \t:\t {mode_minute[0]:.2f} minutes i.e {mode_minute[0]/60:.2f} hours")
            
            # Steps to perform
            print("\r\nData summary : Number of steps (Efforts to prepare) \r\n")
            stepsSummary = self.data['n_steps'].describe()
            print(f"Minimum \t:\t {stepsSummary['min']:.2f} steps")
            print(f"Maximum \t:\t {stepsSummary['max']:.2f} steps")
            print(f"Mean    \t:\t {stepsSummary['mean']:.2f} steps")
            median_steps = self.data['n_steps'].median()
            print(f"Median  \t:\t {median_steps:.2f} steps")
            mode_steps = self.data['n_steps'].mode()
            print(f"Mode    \t:\t {mode_steps[0]:.2f} steps")

            # Ingrediants
            print("\r\nData summary : Required Ingrediants \r\n")
            stepsSummary = self.data['n_ingredients'].describe()
            print(f"Minimum \t:\t {stepsSummary['min']:.2f} ingrediants")
            print(f"Maximum \t:\t {stepsSummary['max']:.2f} ingrediants")
            print(f"Mean    \t:\t {stepsSummary['mean']:.2f} ingrediants")
            median_ing = self.data['n_ingredients'].median()
            print(f"Median  \t:\t {median_ing:.2f} ingrediants")
            mode_ing = self.data['n_ingredients'].mode()
            print(f"Mode    \t:\t {mode_ing[0]:.2f} ingrediants")
        else:
            print("Unable to load Recepe data.")

if __name__ == "__main__":

    # Create an instance of the DataAnalytics class
    recepeData = RawRecipe()
    
    # Load data
    recepeData.load_data()

    # Check for missing values
    recepeData.check_null_values()

    # Get data summary
    recepeData.summarize_dataset()

---> STEP 1 : Loads the data from a CSV file. 

RAW_recipes.csv : Data loaded successfully.
Total Records : 231637 

---> STEP 2 : Null Check for data 

name                 1
id                   0
minutes              0
contributor_id       0
submitted            0
tags                 0
nutrition            0
n_steps              0
steps                0
description       4979
ingredients          0
n_ingredients        0
dtype: int64

---> STEP 3 : Summarize basic statistics for columns 

Data summary : Time to prepare the recepe in minutes 

Minimum 	:	 0.00 minutes
Maximum 	:	 2147483647.00 minutes i.e 35791394.12 hours
Mean    	:	 9398.55 i.e 156.64 hours
Median  	:	 40.00 minutes i.e 0.67 hours
Mode    	:	 30.00 minutes i.e 0.50 hours

Data summary : Number of steps (Efforts to prepare) 

Minimum 	:	 0.00 steps
Maximum 	:	 145.00 steps
Mean    	:	 9.77 steps
Median  	:	 9.00 steps
Mode    	:	 7.00 steps

Data summary : Required Ingrediants 

Minimum 	:	 1.00 ingrediants
Maximum